In [1]:
from framework.Platform import PlatformManager
from framework.Configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True
    prc=True,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

platform_manager.calc_metrics()



Real images found, name:CelebA_Original
3 different generator sources found. Names:
diffusionStyleGAN2
diffusionVAE
guidedDiffusion_IP
[START]: Calculating Metrics for diffusionStyleGAN2
50000


Creating feature extractor "vgg16" with features ['fc2_relu']
Extracting features from input1


KeyboardInterrupt: 

In [2]:
import torch_fidelity
import torch

print(torch.cuda.is_available())

metrics_dict = torch_fidelity.calculate_metrics(
    input1=platform_manager.helper.real_images_src.get_dataset(), 
    input2=platform_manager.helper.get_generator_src("guidedDiffusion_IP").get_dataset(), 
    cuda=True, 
    isc=False, 
    fid=False, 
    kid=False, 
    prc=True, 
    verbose=True,
    save_cpu_ram=False,
)

True


Creating feature extractor "vgg16" with features ['fc2_relu']
Extracting features from input1
Processing samples                                                             
Extracting features from input2
Processing samples                                                             
Precision: 0.54721999168396
Recall: 0.43487998843193054
F-score: 0.4846248416075926


In [4]:
metrics_dict = torch_fidelity.calculate_metrics(
    input1=platform_manager.helper.real_images_src.get_dataset(), 
    input2=platform_manager.helper.get_generator_src("diffusionStyleGAN2").get_dataset(), 
    cuda=True, 
    isc=False, 
    fid=False, 
    kid=False, 
    prc=True, 
    verbose=True,
    save_cpu_ram=False,
)

Creating feature extractor "vgg16" with features ['fc2_relu']
Extracting features from input1
Processing samples                                                             
Extracting features from input2
Processing samples                                                             
Precision: 0.6168400049209595
Recall: 0.3585200011730194
F-score: 0.4534725186722886


In [3]:
from framework.util import img_to_64

img_to_64("D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\cifar-10\\train\\train\\",
             "D:\Dokumente\SoftwareDev\MasterThesis\GIM-evaluation\cifar-10\\64res\\")

(50000, 64, 64, 3) size array saved into celeba64_train.npz
